# Topic:- Waste Classification

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd #for manipulation and analysis
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for data visualization
# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# For Grad-CAM
import tensorflow.keras.backend as K

# Print versions (helps reproducibility)
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


### Set the Path in Python

In [2]:
dataset_dir = r"C:\Users\js640\Downloads\archive (2)\DATASET"
 

### Load Training & Validation Data Using tf.data

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir + "/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir + "/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\js640\\Downloads\\archive (2)\\DATASET/train'

### Print Class Names

In [ ]:
print(train_ds.class_names)


### View Sample Images

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_ds.class_names[labels[i]])
        plt.axis("off")


In [ ]:
#  Save class names
class_names = train_ds.class_names
print("Classes:", class_names)

### Build a CNN Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(224, 224, 3)),  # Normalize

    tf.keras.layers.Conv2D(32, 3, activation='softmax'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, activation='softmax'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, 3, activation='softmax'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='softmax'),
    tf.keras.layers.Dense(len(train_ds.class_names), activation='softmax')
])


In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


### Train the model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Validation accuracy:", accuracy)
print("Loss:", loss)


### Training Accuracy/Loss Plots

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.legend()
plt.show()


In [ ]:
img_path = r"C:\Users\js640\Downloads\archive (2)\DATASET\TEST\R\R_10938.jpg"
#print(os.listdir(img_path))
img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

pred = model.predict(img_array)
class_id = np.argmax(pred)
class_name = train_ds.class_names[class_id]

print("Predicted class:", class_name)


In [ ]:
img_path = r"C:\Users\js640\Downloads\archive (2)\DATASET\TEST\R"
print(os.listdir(img_path))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
# Collect all images and labels from val_ds
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))


In [ ]:
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d",
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Create output folder 
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def show_predictions(n=9):
    plt.figure(figsize=(9,9))
    i = 0
    for images, labels in test_ds.take(5):
        preds = model.predict(images)
        for j in range(images.shape[0]):
            if i >= n: break
            ax = plt.subplot(3,3,i+1)
            img = images[j].numpy()
            img = (img - img.min())/(img.max()-img.min())
            plt.imshow(img)
            true = class_names[int(labels[j])]
            pred = class_names[int(np.argmax(preds[j]))]
            plt.title(f"T:{true}\nP:{pred}")
            plt.axis('off')
            i += 1
        if i >= n: break
    plt.suptitle("Sample predictions")
    plt.savefig(os.path.join(OUTPUT_DIR, "pred_examples.png"))
    plt.show()

show_predictions()

In [ ]:
# Create output folder 
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def show_predictions(n=9):
    plt.figure(figsize=(9,9))
    i = 0
    for images, labels in test_ds.take(1):
        preds = model.predict(images)
        for j in range(images.shape[0]):
            if i >= n: break
            ax = plt.subplot(3,3,i+1)

            img = images[j].numpy()
            img = (img - img.min())/(img.max() - img.min())
            plt.imshow(img)

            true = class_names[int(labels[j])]
            pred = class_names[int(np.argmax(preds[j]))]

            plt.title(f"T:{true}\nP:{pred}")
            plt.axis('off')
            i += 1

    plt.suptitle("Sample predictions")

    # save image
    plt.savefig(os.path.join(OUTPUT_DIR, "pred_examples.png"))
    plt.show()

show_predictions()


In [ ]:

from tensorflow import keras

class_names = ["organic", "recyclable"]

def predict_image(img_path):
    img = keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array / 255.0, axis=0)  # Normalize

    prediction = model.predict(img_array)
    class_id = np.argmax(prediction[0])
    return class_names[class_id]


In [ ]:
print(predict_image(test_dir + "\O\\O_12568.jpg"))

### Save the model

In [ ]:
model.save("waste_classifier.h5")
print("Model saved successfully!")